<a href="https://colab.research.google.com/github/samvardhan777/Colipali/blob/main/Colipali_playground.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install colpali_engine==0.1.1

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.3 MB/s eta 0:00:00
  Created wheel for gputil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=7986785a14a5e86f66baf10b110dc894e2420ba6fa27a8f8a02cbefd0e80fce9
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98f5cbfebc3c397a9d4d0
Successfully built gputil


In [2]:
!pip install "colpali-engine[train]"

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.2/32.2 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 287.4/287.4 kB 24.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.1/249.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 17.1 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
  Attempting uninstall: polars
    Found ex

In [3]:
!pip install pdf2image

In [4]:
from colpali_engine.models import ColPali, ColPaliProcessor

ImportError: cannot import name 'ColPali' from 'colpali_engine.models' (/usr/local/lib/python3.10/dist-packages/colpali_engine/models/__init__.py)

In [5]:
import torch
import typer
from torch.utils.data import DataLoader
from tqdm import tqdm
from transformers import AutoProcessor
from PIL import Image
from typing import List, Tuple, Union

In [6]:
from colpali_engine.models.paligemma_colbert_architecture import ColPali
from colpali_engine.trainer.retrieval_evaluator import CustomEvaluator
from colpali_engine.utils.colpali_processing_utils import process_images, process_queries
from colpali_engine.utils.image_from_page_utils import load_from_dataset

No CUDA runtime is found, using CUDA_HOME='/usr/local/cuda'


In [7]:
from colpali_engine.trainer.retrieval_evaluator import CustomEvaluator

In [8]:
from pdf2image import convert_from_path

In [9]:
!sudo apt-get install poppler-utils

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 186 kB of archives.
After this operation, 696 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 poppler-utils amd64 22.02.0-2ubuntu0.5 [186 kB]
Fetched 186 kB in 1s (207 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 
Selecting previously unselected package poppler-utils.
(Reading database ... 123597 

In [10]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 63.2 MB/s eta 0:00:00


In [11]:
from IPython.display import display

In [12]:
import torch
from typing import List, Union, Tuple
from PIL import Image
import faiss
import numpy as np

MOCK_IMAGE = Image.new("RGB", (448, 448), (255, 255, 255))


def load_model(model_name: str = "vidore/colpaligemma-3b-mix-448-base") -> Tuple[ColPali, AutoProcessor]:
    """Load ColPali model and processor."""
    model = ColPali.from_pretrained(model_name, torch_dtype=torch.bfloat16, device_map="cuda").eval()
    processor = AutoProcessor.from_pretrained(model_name)
    return model, processor


def process_documents(documents: List[Union[str, Image.Image]], processor: AutoProcessor, model: ColPali) -> List[torch.Tensor]:
    """Process documents and get embeddings."""
    dataloader = DataLoader(
        documents,
        batch_size=4,
        shuffle=False,
        collate_fn=lambda x: process_images(processor, x)
    )
    document_embeddings = []
    for batch in tqdm(dataloader):
        with torch.no_grad():
            batch = {k: v.to(model.device) for k, v in batch.items()}
            embeddings = model(**batch)
        document_embeddings.extend(list(torch.unbind(embeddings.to("cpu"))))
    return document_embeddings


def process_queries_func(queries: List[str], processor: AutoProcessor, model: ColPali) -> List[torch.Tensor]:
    """Process queries and get embeddings."""
    dataloader = DataLoader(
        queries,
        batch_size=4,
        shuffle=False,
        collate_fn=lambda x: process_queries(processor, x, MOCK_IMAGE)
    )
    query_embeddings = []
    for batch in dataloader:
        with torch.no_grad():
            batch = {k: v.to(model.device) for k, v in batch.items()}
            embeddings = model(**batch)
        query_embeddings.extend(list(torch.unbind(embeddings.to("cpu"))))
    return query_embeddings


def evaluate_retrieval(query_embeddings: List[torch.Tensor], document_embeddings: np.ndarray, faiss_index: faiss.Index, top_k: int = 5) -> List[Tuple[List[int], List[float]]]:
    """Evaluate the retrieval using FAISS."""

    # Convert query embeddings to float32 numpy array (if they are still in torch.Tensor format)
    if isinstance(query_embeddings, list) and isinstance(query_embeddings[0], torch.Tensor):
        query_embeddings = torch.stack(query_embeddings).float().cpu().numpy()

    # Ensure query embeddings are reshaped to 2D (n_queries, embedding_dim)
    query_embeddings = np.array(query_embeddings)  # Ensure it's a NumPy array
    query_embeddings = query_embeddings.reshape(-1, query_embeddings.shape[-1])

    # Ensure document embeddings are reshaped to 2D (n_documents, embedding_dim)
    if len(document_embeddings.shape) > 2:
        document_embeddings = document_embeddings.reshape(-1, document_embeddings.shape[-1])

    # Print the shapes for debugging
    print(f"Reshaped Query Embeddings Shape: {query_embeddings.shape}")
    print(f"Reshaped Document Embeddings Shape: {document_embeddings.shape}")

    # Add document embeddings to the FAISS index (if not done earlier)
    if faiss_index.ntotal == 0:  # Check if the index is empty
        faiss_index.add(document_embeddings)

    # Perform search with FAISS (top_k is passed as the second positional argument)
    distances, indices = faiss_index.search(query_embeddings, top_k)

    return [(idx_list.tolist(), dist_list.tolist()) for idx_list, dist_list in zip(indices, distances)]


def process_documents_with_filenames(
    documents_with_filenames: List[Tuple[Image.Image, str]],
    processor: AutoProcessor,
    model: ColPali
) -> Tuple[List[torch.Tensor], List[str]]:
    """Process documents and return embeddings along with filenames."""
    images = [doc[0] for doc in documents_with_filenames]  # Extract the image objects
    filenames = [doc[1] for doc in documents_with_filenames]  # Extract the filenames

    dataloader = DataLoader(
        images,
        batch_size=4,
        shuffle=False,
        collate_fn=lambda x: process_images(processor, x)
    )
    document_embeddings = []
    expanded_filenames = []  # Create a new list to hold expanded filenames
    filename_idx = 0  # Track the current filename index

    for batch in tqdm(dataloader):
        with torch.no_grad():
            batch = {k: v.to(model.device) for k, v in batch.items()}
            embeddings = model(**batch)

        document_embeddings.extend(list(torch.unbind(embeddings.to("cpu"))))

        # Expand the filenames list only for the current batch size
        expanded_filenames.extend(filenames[filename_idx:filename_idx + len(batch['input_ids'])])  # Match the current batch
        filename_idx += len(batch['input_ids'])  # Update the filename index to avoid repeating
    print(f"Batch size: {len(batch['input_ids'])}, Embeddings added: {len(document_embeddings)}, Filenames added: {len(expanded_filenames)}")
    return document_embeddings, expanded_filenames




def convert_pdf_to_images(pdf_path: str) -> List[Image.Image]:
    """Convert a PDF into a list of images (one per page)."""
    try:
        images = convert_from_path(pdf_path)
        return images
    except Exception as e:
        print(f"Error converting PDF {pdf_path}: {e}")
        return []


def load_images_with_filenames_from_pdfs(pdf_paths: List[str]) -> List[Tuple[Image.Image, str]]:
    """Convert PDFs to images and return them with filenames."""
    images_with_filenames = []
    for pdf_path in pdf_paths:
        images = convert_pdf_to_images(pdf_path)
        for i, image in enumerate(images):
            image_filename = f"{pdf_path} - Page {i+1}"  # Label each page with the PDF name and page number
            images_with_filenames.append((image, image_filename))
    return images_with_filenames


import torch
import numpy as np
import faiss

def search_documents(
    model,
    processor,
    documents_with_filenames: List[Tuple[Image.Image, str]],
    queries: List[str],
    top_k: int = 5
) -> None:
    """Search and retrieve documents based on queries, showing document names, pages, scores, and images."""

    # Step 1: Process and get embeddings for documents
    document_embeddings, filenames = process_documents_with_filenames(documents_with_filenames, processor, model)

    # Convert document_embeddings to a NumPy array if it's a list of tensors
    if isinstance(document_embeddings, list):
        # If it's a list of tensors, stack them into a single tensor and convert to NumPy
        if isinstance(document_embeddings[0], torch.Tensor):
            document_embeddings = torch.stack(document_embeddings).float().cpu().numpy()

    # Ensure document_embeddings is a NumPy array before accessing shape
    if isinstance(document_embeddings, list):
        document_embeddings = np.array(document_embeddings)

    # Ensure the length of filenames matches the number of document embeddings
    assert document_embeddings.shape[0] == len(filenames), (
        f"Number of document embeddings ({document_embeddings.shape[0]}) does not match number of filenames ({len(filenames)})."
    )

    # Step 2: Process and get embeddings for queries
    query_embeddings = process_queries_func(queries, processor, model)

    print(f"Document Embeddings Shape: {document_embeddings.shape}")
    print(f"Query Embeddings Shape: {query_embeddings[0].shape}")

    # Step 3: Add document embeddings to FAISS index
    faiss_index = add_index(document_embeddings)

    # Step 4: Evaluate retrieval using FAISS
    retrieval_results = evaluate_retrieval(query_embeddings, document_embeddings, faiss_index, top_k)

    print(filenames)
    # Step 5: Display top-k results
    for i, query in enumerate(queries):
        print(f"\nQuery: {query}")
        print(f"Top {top_k} results:")
        top_result_indices, top_result_distances = retrieval_results[i]

        print(top_result_indices)
        print(top_result_distances)



        for rank, (idx, dist) in enumerate(zip(top_result_indices, top_result_distances)):
            if idx >= len(filenames):
                print(f"Warning: Index {idx} is out of range for filenames. Skipping this result.")
                continue  # Skip invalid indices

            # Extract document name and page number from the filename
            try:
                document_name, page_info = filenames[idx].split(' - Page ')
                print(f"Rank {rank + 1}: Document: {document_name}, Page: {page_info}, Distance: {dist:.4f}")
            except Exception as e:
                print(f"Error splitting filename: {filenames[idx]} - {e}")
                continue

            # Display the corresponding image (optional)
            image = documents_with_filenames[idx][0]  # Get the image from the tuple
            display(image)  # Use IPython.display to show the image


def add_index(document_embeddings: np.ndarray) -> faiss.Index:
    """Create and add document embeddings to the FAISS index."""
    # Check the shape and flatten if necessary
    if len(document_embeddings.shape) > 2:
        # Flatten to 2D (e.g., batch size x embedding_dim)
        document_embeddings = document_embeddings.reshape(-1, document_embeddings.shape[-1])

    embedding_dim = document_embeddings.shape[-1]  # Embedding size
    faiss_index = faiss.IndexFlatL2(embedding_dim)

    print(f"Embeddings shape: {document_embeddings.shape}")
    print(f"FAISS index dimensionality: {faiss_index.d}")

    # Add embeddings to the FAISS index
    faiss_index.add(document_embeddings)

    return faiss_index


In [13]:
model, processor = load_model()

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(


model.safetensors.index.json:   0%|          | 0.00/66.3k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/862M [00:00<?, ?B/s]

`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [11]:
pdf_paths = ["/content/364253783.pdf"]

# Convert PDFs to images and get filenames (including page numbers)
documents_with_filenames = load_images_with_filenames_from_pdfs(pdf_paths)

# Queries


In [58]:
documents_with_filenames

[(<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 1'),
 (<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 2'),
 (<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 3'),
 (<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 4'),
 (<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 5'),
 (<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 6')]

In [31]:
documents_with_filenames

[(<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 1'),
 (<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 2'),
 (<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 3'),
 (<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 4'),
 (<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 5'),
 (<PIL.PpmImagePlugin.PpmImageFile image mode=RGB size=1654x2339>,
  '/content/364253783.pdf - Page 6')]

In [55]:
queries = ["Dr.S.Vijayalakshmi", "FINDINGS?"]

# Perform search and show document names with page information
search_documents(model, processor, documents_with_filenames, queries, top_k=3)

100%|██████████| 2/2 [00:16<00:00,  8.10s/it]

Batch size: 2, Embeddings added: 6, Filenames added: 6
Document Embeddings Shape: (6, 1030, 128)
Query Embeddings Shape: torch.Size([16, 128])
Embeddings shape: (6180, 128)
FAISS index dimensionality: 128
Reshaped Query Embeddings Shape: (32, 128)
Reshaped Document Embeddings Shape: (6180, 128)
['/content/364253783.pdf - Page 1', '/content/364253783.pdf - Page 2', '/content/364253783.pdf - Page 3', '/content/364253783.pdf - Page 4', '/content/364253783.pdf - Page 5', '/content/364253783.pdf - Page 6']

Query: Dr.S.Vijayalakshmi
Top 3 results:
[5144, 4114, 3084]
[0.09531795978546143, 0.09844207763671875, 0.1005864143371582]

Query: FINDINGS?
Top 3 results:
[2055, 2057, 1025]
[0.026021122932434082, 0.02623295783996582, 0.02685534954071045]


In [ ]:
queries = ["Frozen Food","XYZ CONSULTING FIRM","Groceries","SOPHIE WRIGHT","Sally Branders"]

In [ ]:
search_documents(model, processor, documents_with_filenames, queries, top_k=3)

100%|██████████| 1/1 [00:09<00:00,  9.67s/it]


tensor([1, 1, 0, 1, 1])
Top 1 Accuracy (verif): 0.2

Query: Frozen Food
Top 3 results:
Document: /content/sample_receipt.png - Score: 15.0625
Document: /content/job-resume-template-259-350x495.png - Score: 15.0
Document: /content/sample_bill.png - Score: 15.0

Query: XYZ CONSULTING FIRM
Top 3 results:
Document: /content/sample_receipt.png - Score: 17.0
Document: /content/job-resume-template-259-350x495.png - Score: 16.875
Document: /content/new-york-resume-templates.png - Score: 16.875

Query: Groceries
Top 3 results:
Document: /content/job-resume-template-259-350x495.png - Score: 14.875
Document: /content/new-york-resume-templates.png - Score: 14.875
Document: /content/sample_receipt.png - Score: 14.875

Query: SOPHIE WRIGHT
Top 3 results:
Document: /content/job-resume-template-259-350x495.png - Score: 16.75
Document: /content/sample_receipt.png - Score: 16.75
Document: /content/new-york-resume-templates.png - Score: 16.625

Query: Sally Branders
Top 3 results:
Document: /content/samp

#  Byaldi

In [ ]:
!git clone https://github.com/AnswerDotAI/byaldi.git

Cloning into 'byaldi'...
remote: Enumerating objects: 148, done.
remote: Counting objects: 100% (148/148), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 148 (delta 74), reused 120 (delta 54), pack-reused 0 (from 0)
Receiving objects: 100% (148/148), 1.97 MiB | 20.58 MiB/s, done.
Resolving deltas: 100% (74/74), done.


In [ ]:
%cd byaldi

/content/byaldi


In [ ]:
!pip install -e "."

Obtaining file:///content/byaldi
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 517.9/517.9 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 307.2/307.2 kB 24.7 MB/s eta 0:00:00
  Building editable for Byaldi (pyproject.toml) ... done
  Created wheel for Byaldi: filename=Byaldi-0.0.2.post2-0.editable-py3-none-any.whl size=14046 sha256=825aca9e0dcce77026cc2095332d2d898cdc6426ac118e98dd5b710b583156a1
  Stored in directory: /tmp/pip-ephem-wheel-cache-_qiyc4x_/wheels/05/35/6c/7376a3a73ed94aa60fd0bd33179ccf86b026e0b8151ba8ba6e
Successfully built Byaldi
  Attempting uninstall: mteb
    Found existing installation: mteb 1.14.15
    Uninstalling mteb-1.14.15:
      Successfully uninstalled mteb-1.14.15


In [ ]:
%cd byaldi

/content/byaldi/byaldi


In [ ]:
from byaldi.byaldi import RAGMultiModalModel

RAG = RAGMultiModalModel.from_pretrained("vidore/colpali")

Verbosity is set to 1 (active). Pass verbose=0 to make quieter.


adapter_config.json:   0%|          | 0.00/752 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/paligemma/configuration_paligemma.py:137: FutureWarning: The `vocab_size` attribute is deprecated and will be removed in v4.44, Please use `text_config.vocab_size` instead.
  warnings.warn(
`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_model.safetensors:   0%|          | 0.00/78.6M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/243k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.26M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.8M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/733 [00:00<?, ?B/s]

In [ ]:
RAG.index(
    input_path="/content/364253783.pdf",
    index_name="image_index", # index will be saved at index_root/index_name/
    store_collection_with_index=False,
    overwrite=True
)

Added page 1 of document 0 to index.
Added page 2 of document 0 to index.
Added page 3 of document 0 to index.
Added page 4 of document 0 to index.
Added page 5 of document 0 to index.
Added page 6 of document 0 to index.
Index exported to .byaldi/image_index
Index exported to .byaldi/image_index


{0: '/content/364253783.pdf'}

In [ ]:
text_query = "Dr.S.Vijayalakshmi? "
results = RAG.search(text_query, k=3)
results

[{'doc_id': 0, 'page_num': 1, 'score': 19.75, 'metadata': {}, 'base64': None},
 {'doc_id': 0, 'page_num': 2, 'score': 16.875, 'metadata': {}, 'base64': None},
 {'doc_id': 0, 'page_num': 6, 'score': 15.125, 'metadata': {}, 'base64': None}]